In [ ]:
# Import Library
# 제출 파일 생성 관련
import os
import zipfile

# 데이터 처리 및 분석
import pandas as pd
import numpy as np
from scipy import stats
from tqdm import tqdm
import seaborn as sns
from matplotlib import pyplot as plt
import datetime

# 머신러닝 전처리
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.model_selection import StratifiedKFold
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline

# 모델 저장
import joblib

# 합성 데이터 생성
from sdv.metadata import SingleTableMetadata
from sdv.single_table import CTGANSynthesizer
from sdv.single_table import TVAESynthesizer
from sdv.evaluation.single_table import run_diagnostic
from sdv.evaluation.single_table import evaluate_quality

# To ignore all warnings
import warnings
warnings.filterwarnings('ignore')


# 데이터 전처리

In [ ]:

# 생성 🏭
# Load Data
train_all = pd.read_csv("./train.csv")
test_all = pd.read_csv("./test.csv")
train = train_all.drop(columns="ID")
train["Fraud_Type"].value_counts()


'\n(*) 리더보드 산식 중 생성데이터의 익명성(TCAP)채점을 위해 각 클래스 별로 1000개의 생성데이터가 반드시 필요합니다.\n(*) 본 베이스 라인에서는 "Fraud_Type" 13종류에 대해 1000개씩 , 총 13,000개의 데이터를 생성할 예정입니다.\n(*) 분류 모델 성능 개선을 위해 생성 데이터를 활용하는 것에는 생성 데이터의 Row 개수에 제한이 없습니다. 단, 리더보드 평가를 위해 제출을 하는 생성 데이터 프레임은 익명성(TCAP) 평가를 위함이며, 위의 조건을 갖춘 생성 데이터를 제출해야합니다.\n'

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120000 entries, 0 to 119999
Data columns (total 63 columns):
 #   Column                                         Non-Null Count   Dtype  
---  ------                                         --------------   -----  
 0   Customer_Birthyear                             120000 non-null  int64  
 1   Customer_Gender                                120000 non-null  object 
 2   Customer_personal_identifier                   120000 non-null  object 
 3   Customer_identification_number                 120000 non-null  object 
 4   Customer_registration_datetime                 120000 non-null  object 
 5   Customer_credit_rating                         120000 non-null  object 
 6   Customer_flag_change_of_authentication_1       120000 non-null  int64  
 7   Customer_flag_change_of_authentication_2       120000 non-null  int64  
 8   Customer_flag_change_of_authentication_3       120000 non-null  int64  
 9   Customer_flag_change_of_authenticatio

In [ ]:
obj_cat_columns=[col for col in train.select_dtypes(['object']).columns if train[col].nunique() <= 20]
obj_idf_columns=[col for col in train.select_dtypes(['object']).columns if train[col].nunique() > 20]
num_cat_columns=[col for col in train.select_dtypes(['int64', 'float64']).columns if train[col].nunique() <= 2]
num_con_columns=[col for col in train.select_dtypes(['int64', 'float64']).columns if train[col].nunique() > 2]

obj_cat_columns.remove('Fraud_Type')

In [ ]:
print('obj_cat_columns : ', obj_cat_columns)
print('obj_idf_columns : ', obj_idf_columns)
print('num_cat_columns : ', num_cat_columns)
print('num_con_columns : ', num_con_columns)

obj_cat_columns :  ['Customer_Gender', 'Customer_credit_rating', 'Customer_loan_type', 'Account_account_type', 'Channel', 'Operating_System', 'Error_Code', 'Type_General_Automatic', 'Access_Medium']
obj_idf_columns :  ['Customer_personal_identifier', 'Customer_identification_number', 'Customer_registration_datetime', 'Account_account_number', 'Account_creation_datetime', 'Transaction_Datetime', 'IP_Address', 'MAC_Address', 'Location', 'Recipient_Account_Number', 'Time_difference', 'Last_atm_transaction_datetime', 'Last_bank_branch_transaction_datetime', 'Transaction_resumed_date']
num_cat_columns :  ['Customer_flag_change_of_authentication_1', 'Customer_flag_change_of_authentication_2', 'Customer_flag_change_of_authentication_3', 'Customer_flag_change_of_authentication_4', 'Customer_rooting_jailbreak_indicator', 'Customer_mobile_roaming_indicator', 'Customer_VPN_Indicator', 'Customer_flag_terminal_malicious_behavior_1', 'Customer_flag_terminal_malicious_behavior_2', 'Customer_flag_term

In [ ]:
import holidays

def is_holiday(date):
    kr_holidays=holidays.KR(years=date.year)
    if date in kr_holidays:
        return 1
    else:
        return 0

def timezone(date):
    if date.hour>=1 and date.hour<=6:
        return 'dawn'

    elif date.hour>=7 and date.hour<=12:
        return 'morning'

    elif date.hour>=13 and date.hour<=18:
        return 'evening'

    else:
        return 'night'

def feature_engineering(df):

  # Time_difference 컬럼을 총 초로 변환 및 이상치 처리
  df['Time_difference_seconds'] = pd.to_timedelta(df['Time_difference']).dt.total_seconds()

  df['Transaction_Datetime']=pd.to_datetime(df['Transaction_Datetime'])
  df['Transaction_Datetime_isholiday']=df['Transaction_Datetime'].apply(is_holiday)
  df['Transaction_Datetime_day']=df['Transaction_Datetime'].apply(lambda x: x.strftime("%A"))
  df['Transaction_Datetime_timezone']=df['Transaction_Datetime'].apply(timezone)

  df['Location_District_Size']=df['Location'].apply(lambda x: 0 if x.split()[0][-1]=="도" else 1)
  df['Location_District']=df['Location'].apply(lambda x: x.split()[0])
  df['Account_after_transaction']=df['Account_balance']-df['Account_initial_balance']

  df['Last_bank_branch_transaction_datetime']=pd.to_datetime(df['Last_bank_branch_transaction_datetime'])
  df['Last_atm_transaction_datetime']=pd.to_datetime(df['Last_atm_transaction_datetime'])
  df['Last_atm_bank_transaction_time_difference']=(df['Last_bank_branch_transaction_datetime']-df['Last_atm_transaction_datetime']).dt.total_seconds()

  df['Customer_malicious_behavior']=(df['Customer_rooting_jailbreak_indicator']+
                                      df['Customer_mobile_roaming_indicator']+
                                      df['Customer_VPN_Indicator']+
                                      df['Customer_flag_terminal_malicious_behavior_1']+
                                      df['Customer_flag_terminal_malicious_behavior_2']+
                                      df['Customer_flag_terminal_malicious_behavior_3']+
                                      df['Customer_flag_terminal_malicious_behavior_4']+
                                      df['Customer_flag_terminal_malicious_behavior_5']+
                                      df['Customer_flag_terminal_malicious_behavior_6'])

  df['Customer_change_of_authentication']=(df['Customer_flag_change_of_authentication_1']+
                                              df['Customer_flag_change_of_authentication_2']+
                                              df['Customer_flag_change_of_authentication_3']+
                                              df['Customer_flag_change_of_authentication_4'])

  df['Account_change']=(df['Account_indicator_release_limit_excess']+
                          df['Account_release_suspention']+
                          df['Flag_deposit_more_than_tenMillion']+
                          df['Unused_account_status']+
                          df['Recipient_account_suspend_status'])

  return df

In [ ]:
train=feature_engineering(train)

In [ ]:

obj_cat_columns=obj_cat_columns+['Location_District', 'Transaction_Datetime_timezone', 'Transaction_Datetime_day']
num_cat_columns=num_cat_columns+['Transaction_Datetime_isholiday', 'Location_District_Size']
num_con_columns=num_con_columns+['Last_atm_bank_transaction_time_difference','Time_difference_seconds', 'Account_after_transaction',
                'Account_change','Customer_change_of_authentication','Customer_malicious_behavior']

# obj_idf_columns  에서 가공할 열 :
# 'Customer_registration_datetime', 'Account_creation_datetime', 'Transaction_Datetime', : 거래 시간(밤,낮인지), 거래 일자(공휴일 혹은 평일)
# 'Location', : 이상 거래가 자주 발생하는 장소 - 시군구로 나누고 시각화해보기
# 'Last_atm_transaction_datetime', 'Last_bank_branch_transaction_datetime', 'Transaction_resumed_date' : 연도-월로 나눠 가장 최근으로부터 얼마나 떨어져 있는지 확인

In [ ]:
print('obj_cat_columns : ', obj_cat_columns)
print('obj_idf_columns : ', obj_idf_columns)
print('num_cat_columns : ', num_cat_columns)
print('num_con_columns : ', num_con_columns)

obj_cat_columns :  ['Customer_Gender', 'Customer_credit_rating', 'Customer_loan_type', 'Account_account_type', 'Channel', 'Operating_System', 'Error_Code', 'Type_General_Automatic', 'Access_Medium', 'Location_District', 'Transaction_Datetime_timezone', 'Transaction_Datetime_day']
obj_idf_columns :  ['Customer_personal_identifier', 'Customer_identification_number', 'Customer_registration_datetime', 'Account_account_number', 'Account_creation_datetime', 'Transaction_Datetime', 'IP_Address', 'MAC_Address', 'Location', 'Recipient_Account_Number', 'Time_difference', 'Last_atm_transaction_datetime', 'Last_bank_branch_transaction_datetime', 'Transaction_resumed_date']
num_cat_columns :  ['Customer_flag_change_of_authentication_1', 'Customer_flag_change_of_authentication_2', 'Customer_flag_change_of_authentication_3', 'Customer_flag_change_of_authentication_4', 'Customer_rooting_jailbreak_indicator', 'Customer_mobile_roaming_indicator', 'Customer_VPN_Indicator', 'Customer_flag_terminal_malicio

In [ ]:
model_fit_columns=obj_cat_columns+num_cat_columns+num_con_columns
# model_fit_columns=[x for x in model_fit_columns if x not in ['ID', 'Fraud_Type']]

# EDA

# 데이터 생성

In [ ]:
fraud_types = train['Fraud_Type'].unique()

N_CLS_PER_GEN = 500
N_SAMPLE = 100

cat_esb = CategoricalEnsemble()

train_new = train_all.drop(columns="ID")
train_new['Time_difference_seconds'] = pd.to_timedelta(train_new['Time_difference']).dt.total_seconds()

In [ ]:
# 모든 합성 데이터를 저장할 DataFrame 초기화
all_synthetic_data_ctgan = pd.DataFrame()

# 각 Fraud_Type에 대해 합성 데이터 생성 및 저장
for fraud_type in tqdm(fraud_types):

    # 해당 Fraud_Type에 대한 서브셋 생성
    subset = train_new[train_new["Fraud_Type"] == fraud_type]

    # 모든 Fraud_Type에 대해 100개씩 샘플링
    subset = subset.sample(n=N_SAMPLE, random_state=42)

    # Time_difference 열 제외 (초 단위로 변환된 컬럼만 사용)
    subset = subset.drop('Time_difference', axis=1)

    # 메타데이터 생성 및 모델 학습
    metadata = SingleTableMetadata()

    metadata.detect_from_dataframe(subset)
    metadata.set_primary_key(None)

    # 데이터 타입 설정
    column_sdtypes = {
        'Account_initial_balance': 'numerical',
        'Account_balance': 'numerical',
        'Customer_identification_number': 'categorical',
        'Customer_personal_identifier': 'categorical',
        'Account_account_number': 'categorical',
        'IP_Address': 'ipv4_address',
        'Location': 'categorical',
        'Recipient_Account_Number': 'categorical',
        'Fraud_Type': 'categorical',
        'Time_difference_seconds': 'numerical',
        'Customer_Birthyear': 'numerical'
    }

    # 각 컬럼에 대해 데이터 타입 설정
    for column, sdtype in column_sdtypes.items():
        metadata.update_column(
            column_name=column,
            sdtype=sdtype
        )

    synthesizer = CTGANSynthesizer(
                            metadata,
                            epochs=100
                        )
    synthesizer.fit(subset)

    synthetic_subset = synthesizer.sample(num_rows=N_CLS_PER_GEN)

    # 생성된 데이터를 all_synthetic_data_ctgan에 추가
    all_synthetic_data_ctgan = pd.concat([all_synthetic_data_ctgan, synthetic_subset], ignore_index=True)



100%|██████████| 13/13 [13:08<00:00, 60.67s/it]


In [ ]:
all_synthetic_data_ctgan.columns

Index(['Customer_Birthyear', 'Customer_Gender', 'Customer_personal_identifier',
       'Customer_identification_number', 'Customer_registration_datetime',
       'Customer_credit_rating', 'Customer_flag_change_of_authentication_1',
       'Customer_flag_change_of_authentication_2',
       'Customer_flag_change_of_authentication_3',
       'Customer_flag_change_of_authentication_4',
       'Customer_rooting_jailbreak_indicator',
       'Customer_mobile_roaming_indicator', 'Customer_VPN_Indicator',
       'Customer_loan_type', 'Customer_flag_terminal_malicious_behavior_1',
       'Customer_flag_terminal_malicious_behavior_2',
       'Customer_flag_terminal_malicious_behavior_3',
       'Customer_flag_terminal_malicious_behavior_4',
       'Customer_flag_terminal_malicious_behavior_5',
       'Customer_flag_terminal_malicious_behavior_6',
       'Customer_inquery_atm_limit', 'Customer_increase_atm_limit',
       'Account_account_number', 'Account_account_type',
       'Account_creation_da

In [ ]:
train_new.columns

Index(['Customer_Birthyear', 'Customer_Gender', 'Customer_personal_identifier',
       'Customer_identification_number', 'Customer_registration_datetime',
       'Customer_credit_rating', 'Customer_flag_change_of_authentication_1',
       'Customer_flag_change_of_authentication_2',
       'Customer_flag_change_of_authentication_3',
       'Customer_flag_change_of_authentication_4',
       'Customer_rooting_jailbreak_indicator',
       'Customer_mobile_roaming_indicator', 'Customer_VPN_Indicator',
       'Customer_loan_type', 'Customer_flag_terminal_malicious_behavior_1',
       'Customer_flag_terminal_malicious_behavior_2',
       'Customer_flag_terminal_malicious_behavior_3',
       'Customer_flag_terminal_malicious_behavior_4',
       'Customer_flag_terminal_malicious_behavior_5',
       'Customer_flag_terminal_malicious_behavior_6',
       'Customer_inquery_atm_limit', 'Customer_increase_atm_limit',
       'Account_account_number', 'Account_account_type',
       'Account_creation_da

In [ ]:
diagnostic_report = run_diagnostic(
    real_data=train_new.drop(columns=['Time_difference']),
    synthetic_data=all_synthetic_data_ctgan,
    metadata=metadata)

diagnostic_report.get_details(property_name='Data Validity')
diagnostic_report.get_details(property_name='Data Structure')


quality_report = evaluate_quality(
    real_data=train_new.drop(columns=['Time_difference']),
    synthetic_data=all_synthetic_data_ctgan,
    metadata=metadata)

quality_report.get_details(property_name='Column Shapes')
quality_report.get_details(property_name='Column Pair Trends')

quality_report.save(filepath=f'results/{datetime.now()}_quality_report.pkl')
diagnostic_report.save(filepath=f'results/{datetime.now()}_diagnostic_report.pkl')

synthesizer.save(
    filepath=f'{datetime.now()}_ctgan_synthesizer.pkl'
)

fig = synthesizer.get_loss_values_plot()
fig.show()

Generating report ...

(1/2) Evaluating Data Validity: |██████████| 63/63 [00:00<00:00, 128.72it/s]|
Data Validity Score: 99.92%

(2/2) Evaluating Data Structure: |██████████| 1/1 [00:00<00:00, 194.48it/s]|
Data Structure Score: 100.0%

Overall Score (Average): 99.96%

Generating report ...

(1/2) Evaluating Column Shapes: |██████████| 63/63 [00:35<00:00,  1.78it/s]|
Column Shapes Score: 84.07%

(2/2) Evaluating Column Pair Trends: |▋         | 143/1953 [01:20<1:37:19,  3.23s/it]|

KeyboardInterrupt: 

In [ ]:
# 모든 합성 데이터를 저장할 DataFrame 초기화
all_synthetic_data_tvae = pd.DataFrame()

# 각 Fraud_Type에 대해 합성 데이터 생성 및 저장
for fraud_type in tqdm(fraud_types):

    # 해당 Fraud_Type에 대한 서브셋 생성
    subset = train_new[train_new["Fraud_Type"] == fraud_type]

    # 모든 Fraud_Type에 대해 100개씩 샘플링
    subset = subset.sample(n=N_SAMPLE, random_state=42)

    # Time_difference 열 제외 (초 단위로 변환된 컬럼만 사용)
    subset = subset.drop('Time_difference', axis=1)

    # 메타데이터 생성 및 모델 학습
    metadata = SingleTableMetadata()

    metadata.detect_from_dataframe(subset)
    metadata.set_primary_key(None)

    # 데이터 타입 설정
    column_sdtypes = {
        'Account_initial_balance': 'numerical',
        'Account_balance': 'numerical',
        'Customer_identification_number': 'categorical',
        'Customer_personal_identifier': 'categorical',
        'Account_account_number': 'categorical',
        'IP_Address': 'ipv4_address',
        'Location': 'categorical',
        'Recipient_Account_Number': 'categorical',
        'Fraud_Type': 'categorical',
        'Time_difference_seconds': 'numerical',
        'Customer_Birthyear': 'numerical'
    }

    # 각 컬럼에 대해 데이터 타입 설정
    for column, sdtype in column_sdtypes.items():
        metadata.update_column(
            column_name=column,
            sdtype=sdtype
        )

    synthesizer = TVAESynthesizer(
                            metadata,
                            epochs=100
                        )
    synthesizer.fit(subset)

    synthetic_subset = synthesizer.sample(num_rows=N_CLS_PER_GEN)


    # 생성된 데이터를 all_synthetic_data에 추가
    all_synthetic_data_tvae = pd.concat([all_synthetic_data_tvae, synthetic_subset], ignore_index=True)


In [ ]:
diagnostic_report = run_diagnostic(
    real_data=train_new.drop(columns=['Time_difference']),
    synthetic_data=all_synthetic_data_tvae,
    metadata=metadata)

diagnostic_report.get_details(property_name='Data Validity')
diagnostic_report.get_details(property_name='Data Structure')


quality_report = evaluate_quality(
    real_data=train_new.drop(columns=['Time_difference']),
    synthetic_data=all_synthetic_data_tvae,
    metadata=metadata)

quality_report.get_details(property_name='Column Shapes')
quality_report.get_details(property_name='Column Pair Trends')

quality_report.save(filepath=f'results/{datetime.now()}_quality_report.pkl')
diagnostic_report.save(filepath=f'results/{datetime.now()}_diagnostic_report.pkl')

synthesizer.save(
    filepath=f'{datetime.now()}_tvae_synthesizer.pkl'
)datetime

fig = synthesizer.get_loss_values_plot()
fig.show()